Book-Crossing: This dataset was collected by Cai-Nicolas Ziegler in a 4-week crawl (August / September 2004) from the Book-Crossing community with kind permission from Ron Hornbaker, CTO of Humankind Systems. It contains 278,858 users (anonymized but with demographic information) providing 1,149,780 ratings (explicit / implicit) about 271,379 books.

Book-Crossing Dataset:
- Split based on the implicit and explicit feedback
- Remove users with fewer than 5 interaction

In [1]:
# importing required pakcages
import pandas as pd
from tqdm import tqdm

In [2]:
# config
data_config = 'Bookcrossing'
config = {
    'Bookcrossing':
    {
        'ds_name': 'Bookcrossing',
        'ds_fullname': 'BX-Book-Ratings',
        'ds_fmt': 'csv',
        'ds_acronym': 'BX',
        'ds_rate': 5,
        'ds_toprate': 200,
        'ds_sep': ';',
        'ds_encode': 'ISO-8859–1'
    }
}

In [3]:
# loading the Book-Crossing dataset
dataset = pd.read_csv(f"dataset/{data_config}/BX-Book-Ratings.csv", sep=f"{config[data_config]['ds_sep']}", encoding=f"{config[data_config]['ds_encode']}")
dataset.columns = ['User-ID', 'Book-ID', 'Book-Rating']

In [4]:
dataset.head()

,User-ID,Book-ID,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
# what columns or attributes the dataset is included?
dataset.columns

Index(['User-ID', 'Book-ID', 'Book-Rating'], dtype='object')

In [6]:
# statistics on explicit dataset
print("Original dataset statistics: ")
print(f"> No. of users: {len(dataset['User-ID'].unique())}")
print(f"> No. of Books: {len(dataset['Book-ID'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Original dataset statistics: 
> No. of users: 105283
> No. of Books: 340556
> No. of Interaction: 1149780


In [13]:
# checking the number of times in which a rating happened - (0: implicit, 1-10: explicit)
dataset['Book-Rating'].value_counts()

0     716109
8     103736
10     78610
7      76457
9      67541
5      50974
6      36924
4       8904
3       5996
2       2759
1       1770
Name: Book-Rating, dtype: int64

In [14]:
# This method return a dataframe in which the specified values are removed from a specific column. We can use it to generate the implicit/explicit dataset.
# To do this, we can remove the 0 or 1-10 values from the `Book-Rating` column.
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [21]:
# To create the explicit dataset, we first remove those rows which includes 0 on Book-Rating columns (implicit ones), then we save the new generated daaset.
if data_config == 'Bookcrossing':
    dataset = filter_rows_by_values(dataset, "Book-Rating", [0])

    print(dataset['Book-Rating'].value_counts())

    # statistics on explicit dataset
    print("Explicit dataset statistics: ")
    print(f"> No. of users: {len(dataset['User-ID'].unique())}")
    print(f"> No. of Books: {len(dataset['Book-ID'].unique())}")
    print(f"> No. of Interaction: {dataset.shape[0]}")

8     103736
10     78610
7      76457
9      67541
5      50974
6      36924
4       8904
3       5996
2       2759
1       1770
Name: Book-Rating, dtype: int64
Explicit dataset statistics: 
> No. of users: 77805
> No. of Books: 185973
> No. of Interaction: 433671


### Remove user with more than TopRate interactions

In [22]:
# To check if there is any user with more than toprate interaction in the preprocessed dataset
# The correct output will be zero
uid_value_counts = dataset['User-ID'].value_counts()
print(f"The numebr of users with more than {config[data_config]['ds_toprate']} interactions: {uid_value_counts[uid_value_counts > config[data_config]['ds_toprate']].count()}")

The numebr of users with more than 200 interactions: 144


In [23]:
# To remove the users with fewer than 5 interaction we first count the number of interactino per user and add a new column (`Count`) in the dataframe.
# This column shows the number of interaction per user in the dataset
users_counts = dataset['User-ID'].value_counts()
users_counts = users_counts.to_dict() #converts to dictionary
dataset['Count'] = dataset['User-ID'].map(users_counts)

In [24]:
dataset = filter_rows_by_values(dataset, "Count", list(range(config[data_config]['ds_toprate'], max(dataset['Count']) + 1)))

In [25]:
# statistics on explicit dataset after removing users with more than 200 int.
print(f"New dataset statistics (users with interactions < {config[data_config]['ds_toprate']}): ")
print(f"> No. of users: {len(dataset['User-ID'].unique())}")
print(f"> No. of Books: {len(dataset['Book-ID'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

New dataset statistics (users with interactions < 200): 
> No. of users: 77660
> No. of Books: 156891
> No. of Interaction: 364245


### Iteratively remove users and items with fewer than `ds_rate` interactions

In [26]:
list(range(config[data_config]['ds_rate']))

[0, 1, 2, 3, 4]

In [27]:
user_interaction, item_interaction = 1, 1

while user_interaction != 0 or item_interaction != 0:
    print(f"The current number of user and item with < {config[data_config]['ds_rate']} interactions: ")
    # user side fewer than ds_rate cheking
    uid_value_counts = dataset['User-ID'].value_counts()
    user_interaction = uid_value_counts[uid_value_counts < config[data_config]['ds_rate']].count()
    print(f"No. of users < {config[data_config]['ds_rate']} ineractions: {user_interaction}")

    users_counts = dataset['User-ID'].value_counts()
    users_counts = users_counts.to_dict() #converts to dictionary
    dataset['Count'] = dataset['User-ID'].map(users_counts)

    dataset = filter_rows_by_values(dataset, "Count", list(range(config[data_config]['ds_rate'])))

    # item side fewer than ds_rate cheking
    bid_value_counts = dataset['Book-ID'].value_counts()
    item_interaction = bid_value_counts[bid_value_counts < config[data_config]['ds_rate']].count()
    print(f"No. of items < {config[data_config]['ds_rate']} ineractions: {item_interaction}")

    items_counts = dataset['Book-ID'].value_counts()
    items_counts = items_counts.to_dict() #converts to dictionary
    dataset['Count'] = dataset['Book-ID'].map(items_counts)

    dataset = filter_rows_by_values(dataset, "Count", list(range(config[data_config]['ds_rate'])))

The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 63585
No. of items < 5 ineractions: 119254
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 5825
No. of items < 5 ineractions: 1594
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 432
No. of items < 5 ineractions: 208
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 69
No. of items < 5 ineractions: 45
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 19
No. of items < 5 ineractions: 17
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 11
No. of items < 5 ineractions: 13
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 6
No. of items < 5 ineractions: 2
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 2
No. of items < 

In [28]:
# statistics on 5 rate explicit dataset (after pre-processing)
print(f"No. of users: {len(dataset['User-ID'].unique())}")
print(f"No. of Books: {len(dataset['Book-ID'].unique())}")
print(f"No. of Interaction: {dataset.shape[0]}")

No. of users: 6358
No. of Books: 6921
No. of Interaction: 88552


In [29]:
# Before we save the preprocessed explicit dataset (5Rate) we first remove the added column which is `Count`
del dataset['Count']

### User and Item ID Mapping

In [30]:
uid_to_index = dict()
bid_to_index = dict()

last_user_id = 0
last_book_id = 0

for eachline in tqdm(dataset.iterrows()):
  # add a new user id with an index
  if eachline[1][0] not in uid_to_index.keys():
    uid_to_index[eachline[1][0]] = last_user_id
    last_user_id += 1
  # add a new book id with an index
  if eachline[1][1] not in bid_to_index.keys():
    bid_to_index[eachline[1][1]] = last_book_id
    last_book_id += 1

88552it [00:11, 7942.54it/s]


In [32]:
# write the file with new mapped indices into a txt file
mapped_dataset = open(f"dataset/{data_config}/{config[data_config]['ds_acronym']}-Book-Explicit-{config[data_config]['ds_rate']}Rate-Map.csv", 'w')
mapped_dataset_txt = open(f"dataset/{data_config}/{config[data_config]['ds_acronym']}-Book-Explicit-{config[data_config]['ds_rate']}Rate-Map.txt", 'w')
mapped_dataset.write('uid' + ',' + 'bid' + ',' + 'rating' + '\n')

for eachline in tqdm(dataset.iterrows()):
    mapped_dataset.write(str(uid_to_index[eachline[1][0]]) + "," + str(bid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
    mapped_dataset_txt.write(str(uid_to_index[eachline[1][0]]) + "," + str(bid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
mapped_dataset.close()

88552it [00:12, 6968.26it/s]
